In [9]:
import numpy as np
import random
import networkx as nx
import pickle
import time
from scipy import sparse
import torch
from tqdm.auto import trange
from xx_AllFunctions import gnp_random_connected_graph, get_adjacency_feature_centrality,distance_sum, get_adjacency_centrality, get_node_embedding, sparse_mx_to_torch_sparse_tensor

In [13]:
def damage_graph(base_graph):
    G0=base_graph.copy()
    while True:
        tempg=base_graph.copy()
        ndel=list(np.random.randint(0,len(base_graph.edges())-1,random.randint(0,20)))
        ndel.sort(reverse=True)
        # ndel=i
        try:
            for j in range(len(ndel)):
                u,v=list(base_graph.edges())[ndel[j]][0:2]
                
                tempg.remove_edge(u, v)
            
            if nx.is_connected(tempg):
                for j in range(len(ndel)):
                    u,v=list(base_graph.edges())[ndel[j]][0:2]
                    # G0.remove_edge(u, v)
                    G0.edges[u,v]['weight'] = 1000000000
                break
            else:
                print("try again")
                continue
        except KeyError:
            continue
        except  nx.NetworkXError:
            continue
    # list_node_weights_train.append(np.random.random(max_node).reshape(1,-1))
    return G0.to_directed()

def get_fm_centrality(graph: nx.Graph):
    pfilename="test.edg"

    # ebc=nx.betweenness_centrality(g, weight = 'weight')
    total_sum_new=distance_sum(graph)
    # eff=total_sum_orig/total_sum_new
    
    centrality = total_sum_new
        
    # adj_mat = np.zeros((num_nodes,num_nodes))
    # adj_mat_excerpt = get_adj_matrix(g).todense()
    # adj_mat[0:adj_mat_excerpt.shape[0], 0:adj_mat_excerpt.shape[1]] = adj_mat_excerpt
    # adj_mat = sparse.csr_matrix(adj_mat)
    

    
    ndim = 256
    feature_mat = np.zeros((graph.number_of_nodes(),ndim))
    feature_mat_excerpt = get_node_embedding(graph, dimensions = ndim, walk_length = 50, num_walks = 50, workers = 12,pfilename=pfilename).todense()
    feature_mat[0:feature_mat_excerpt.shape[0]] = feature_mat_excerpt
    feature_mat = sparse.csr_matrix(feature_mat)

    # adj_mat = sparse_mx_to_torch_sparse_tensor(adj_mat)  

    feature_mat = sparse_mx_to_torch_sparse_tensor(feature_mat)

    edge_index = torch.tensor(list(graph.edges)).t().contiguous()
    edge_weight = torch.tensor([graph[u][v]['weight'] for u, v in graph.edges])

    return edge_index, edge_weight, centrality, feature_mat

def get_centrality(graph):
    # ebc=nx.betweenness_centrality(g, weight = 'weight')
    total_sum_new=distance_sum(graph)
    # eff=total_sum_orig/total_sum_new
    centrality = total_sum_new
        
    # adj_mat = np.zeros((num_nodes,num_nodes))
    # adj_mat_excerpt = get_adj_matrix(g).todense()
    # adj_mat[0:adj_mat_excerpt.shape[0], 0:adj_mat_excerpt.shape[1]] = adj_mat_excerpt
    # adj_mat = sparse.csr_matrix(adj_mat)
    
    # adj_mat = sparse_mx_to_torch_sparse_tensor(adj_mat)  
    # list_adj.append(adj_mat)
    
    edge_index = torch.tensor(list(graph.edges)).t().contiguous()
    edge_weight = torch.tensor([graph[u][v]['weight'] for u, v in graph.edges])

    return edge_index, edge_weight, centrality

In [14]:
class GNNSample:
    def __init__(self, base_graph):
        self.base_graph = base_graph
        self.mod_graph = damage_graph(base_graph)

        self.base_edge_index, self.base_edge_weight, self.base_centrality, self.base_fm = get_fm_centrality(self.base_graph)
        self.mod_edge_index, self.mod_edge_weight, self.mod_centrality = get_centrality(self.mod_graph)

In [15]:
n_train_g = 100

nnodes = 500

prob_edge = 1.2/(100-1)

list_train_graph_orig = []
list_train_graph = list()

list_test_graph_orig  = []
list_test_graph= list()

new_graph = gnp_random_connected_graph(nnodes, prob_edge)
graphs = [GNNSample(new_graph) for _ in trange(n_train_g)]

  0%|          | 0/100 [00:00<?, ?it/s]

Edges: 2025
Extend=True


c:\Users\Anderson\miniconda3\envs\gnn\lib\site-packages\pecanpy\rw\sparse_rw.py:30: RuntimeWarning: Mean of empty slice.
  data[indptr[i] : indptr[i + 1]].mean()
c:\Users\Anderson\miniconda3\envs\gnn\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\Anderson\miniconda3\envs\gnn\lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\Anderson\miniconda3\envs\gnn\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\Anderson\miniconda3\envs\gnn\lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/5000 [00:00<?, ?it/s]

Pecanpy:8.448895692825317


...

  0%|          | 0/5000 [00:00<?, ?it/s]

Pecanpy:4.167032957077026
Edges: 2025
Extend=True


c:\Users\Anderson\miniconda3\envs\gnn\lib\site-packages\pecanpy\rw\sparse_rw.py:30: RuntimeWarning: Mean of empty slice.
  data[indptr[i] : indptr[i + 1]].mean()
c:\Users\Anderson\miniconda3\envs\gnn\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\Anderson\miniconda3\envs\gnn\lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\Anderson\miniconda3\envs\gnn\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\Anderson\miniconda3\envs\gnn\lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/5000 [00:00<?, ?it/s]

Pecanpy:4.199702739715576


In [16]:
for sample in graphs:
    sample.ratio = 1 - (sample.mod_centrality - sample.base_centrality) / sample.base_centrality

ratio_min = min(graphs, key=lambda sample: sample.ratio).ratio

for sample in graphs:
    sample.ratio_normalized = (sample.ratio - ratio_min) / (1 - ratio_min)

In [5]:
max_model_no = len(new_graph.nodes)*1



pfilename="orig1.edg"


list_edge_index_train_orig,list_edge_weight_train_orig, eff_mat_train_orig, list_fm_train_orig = get_adjacency_feature_centrality(list_train_graph_orig, max_model_no,pfilename=pfilename)#,total_sum_orig)
list_edge_index_test_orig,list_edge_weight_test_orig, eff_mat_test_orig, list_fm_test_orig = get_adjacency_feature_centrality(list_test_graph_orig, max_model_no,pfilename=pfilename)#,total_sum_orig)
  
list_edge_index_train,list_edge_weight_train, eff_mat_train= get_adjacency_centrality(list_train_graph, max_model_no)#,total_sum_orig)
list_edge_index_test,list_edge_weight_test, eff_mat_test = get_adjacency_centrality(list_test_graph, max_model_no)#,total_sum_orig)


eff_mat_final_train=1-(eff_mat_train-eff_mat_train_orig)/eff_mat_train_orig
eff_mat_final_test=1-(eff_mat_test-eff_mat_test_orig)/eff_mat_test_orig



eff_mat_final_complete=np.concatenate([eff_mat_final_train,eff_mat_final_test],axis=1)



eff_mat_final_complete_scaled = (eff_mat_final_complete - np.min(eff_mat_final_complete)) / (1 - np.min(eff_mat_final_complete))




eff_mat_final_train_scaled=eff_mat_final_complete_scaled[0,0:2000].reshape(1,-1)
eff_mat_final_test_scaled=eff_mat_final_complete_scaled[0,2000:].reshape(1,-1)

Generating Graph: 0
Max_Node: 500
EFF:23.63743257522583
Edges: 4070
Extend=True


  0%|          | 0/5000 [00:00<?, ?it/s]

Pecanpy:17.872640371322632
Generating Graph: 0
Max_Edge: 500


c:\Users\Anderson\Desktop\gnn-transport\single\xx_AllFunctions.py:306: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:620.)
  return torch.sparse.FloatTensor(indices, values, shape)


EFF:23.158284902572632
Generating Graph: 1
Max_Edge: 500
EFF:24.310339212417603
Generating Graph: 2
Max_Edge: 500
EFF:25.143789529800415
Generating Graph: 3
Max_Edge: 500
EFF:24.45358896255493
Generating Graph: 4
Max_Edge: 500
EFF:24.400417804718018
Generating Graph: 5
Max_Edge: 500
EFF:24.633171558380127
Generating Graph: 6
Max_Edge: 500
EFF:24.020379543304443
Generating Graph: 7
Max_Edge: 500
EFF:24.260612726211548
Generating Graph: 8
Max_Edge: 500
EFF:23.748554468154907
Generating Graph: 9
Max_Edge: 500
EFF:24.029935598373413
Generating Graph: 10
Max_Edge: 500
EFF:24.266023874282837
Generating Graph: 11
Max_Edge: 500
EFF:23.340346574783325
Generating Graph: 12
Max_Edge: 500
EFF:24.22432827949524
Generating Graph: 13
Max_Edge: 500
EFF:24.360738277435303
Generating Graph: 14
Max_Edge: 500
EFF:24.108261585235596
Generating Graph: 15
Max_Edge: 500
EFF:24.219799041748047
Generating Graph: 16
Max_Edge: 500
EFF:24.16679620742798
Generating Graph: 17
Max_Edge: 500
EFF:24.31426429748535
Gene

In [18]:
with open(r'./graph_data_train_class.pickle', 'wb') as handle:
    pickle.dump(graphs, handle, protocol=pickle.HIGHEST_PROTOCOL)